##**Install Packages**

In [ ]:
!python --version
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local

!conda init
!conda install -q python=3.9
!conda create -q --name py39 python=3.9
!source  activate py39
!python --version


In [ ]:
!pip install lightning-flash

In [ ]:
!pip install 'lightning-flash[image]'

In [ ]:
!pip install segmentation_models_pytorch
!pip install tensorboard==2.9.1  # หรือเวอร์ชันที่เหมาะสม
!pip install setuptools==59.5.0

##**Download DataSet**




In [2]:
%%writefile download.py
from icevision.all import *
import os
import shutil
import glob
data_url = 'https://s3.amazonaws.com/fast-ai-sample/camvid_tiny.tgz'
data = icedata.load_data(data_url,"data")


destination_dir = os.getcwd()  # ใช้ไดเรกทอรีปัจจุบันในการทำงาน

# เคลื่อนย้ายโฟลเดอร์ไปยังไดเรกทอรีปลายทาง
shutil.move(str(data), os.path.join(destination_dir, 'data'))


print(data)

Writing download.py


In [ ]:
!python download.py

##**Setup-dataset**

In [ ]:

import os

folder_path = "/content/data/camvid_tiny/labels"  # เปลี่ยนเป็น path ของโฟลเดอร์ของคุณ

for filename in os.listdir(folder_path):
    if filename.endswith("_P.png"):
        new_filename = filename.replace("_P", "")
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")


In [9]:
%%writefile check.py
import flash
import torch
from flash.core.data.utils import download_data
from flash.image import SemanticSegmentation, SemanticSegmentationData
from pytorch_lightning.loggers import CSVLogger
import os
from icevision.all import *
import numpy as np
codes = np.loadtxt(os.path.join(r"/content/data/camvid_tiny","codes.txt"),dtype=str)
print(codes)
class_map = ClassMap(list(codes))
print(class_map)
print(class_map.num_classes)

Overwriting check.py


In [ ]:
!python check.py

##**Train Model**

In [63]:
%%writefile train.py

import flash
import torch
from flash.core.data.utils import download_data
from flash.image import SemanticSegmentation, SemanticSegmentationData
from pytorch_lightning.loggers import CSVLogger
import os


from icevision.all import *
import numpy as np
codes = np.loadtxt(os.path.join(r"/content/data/camvid_tiny","codes.txt"),dtype=str)
class_map = ClassMap(list(codes))

# 1. Create the DataModule
# The data was generated with the  CARLA self-driving simulator as part of the Kaggle Lyft Udacity Challenge.
# More info here: https://www.kaggle.com/kumaresanmanickavelu/lyft-udacity-challenge
# download_data(
#     "https://github.com/ongchinkiat/LyftPerceptionChallenge/releases/download/v0.1/carla-capture-20180513A.zip",
#     "./data",
# )
print(SemanticSegmentation.available_backbones('unet'))

img_path = os.path.join(r"/content/data/camvid_tiny/images")
label_path = os.path.join(r"/content/data/camvid_tiny/labels")

datamodule = SemanticSegmentationData.from_folders(
    train_folder=img_path,
    train_target_folder=label_path,
    val_split=0.1,
    transform_kwargs={"image_size": (256, 256)},
    num_classes=class_map.num_classes,
    batch_size=4,
)

# 2. Build the task
model = SemanticSegmentation(
    backbone="mobilenetv3_large_100",
    head="fpn",
    num_classes=datamodule.num_classes,
)
print("Processing training")
trainer = flash.Trainer(
    max_epochs=50,
    logger=CSVLogger(save_dir='logs/'),
    gpus=torch.cuda.device_count(),
    precision=16 if torch.cuda.device_count() else 32,
#     limit_train_batches=0.1,
#     limit_val_batches=0.1,
)
trainer.finetune(model, datamodule=datamodule, strategy="freeze")
trainer.save_checkpoint("semantic_segmentation_model_model_type_fpn_bb_mobilenetv3_large_100.pt")




Overwriting train.py


In [ ]:
!python train.py

##**Evaluate Model**

In [69]:
%%writefile ploy.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# อ่านข้อมูลจาก CSV
csv_path = 'logs/lightning_logs/version_4/metrics.csv'
metrics = pd.read_csv(f'{csv_path}')
del metrics["step"]
metrics.set_index("epoch", inplace=True)

# สร้างกราฟเส้นโดยไม่มีแถบความเชื่อมั่น
g = sns.relplot(data=metrics, kind="line", errorbar=None)
g.fig.set_size_inches(12, 6)  # ปรับขนาดกราฟ
plt.grid()  # เพิ่มกริด

# บันทึกเป็นไฟล์ PNG
plt.savefig("metrics_plot.png")
# หรือบันทึกเป็นไฟล์ PDF
# plt.savefig("metrics_plot.pdf")



Overwriting ploy.py


In [70]:
!python ploy.py

##**Test Predict**

In [65]:
%%writefile test.py
import flash
from flash.image import SemanticSegmentation, SemanticSegmentationData
from flash import Trainer
from flash.core.data.io.input import DataKeys
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os, glob

# Load the model
model = SemanticSegmentation.load_from_checkpoint("semantic_segmentation_model_model_type_fpn_bb_mobilenetv3_large_100.pt")

trainer = Trainer(
    gpus=torch.cuda.device_count(),  # Check the number of available GPUs
    precision=16 if torch.cuda.device_count() else 32  # Use precision 16 if GPU is available
)

# Sample images
sample_imgs1 = []
sample_imgs2 = glob.glob("/content/data/camvid_tiny/images/*.png")[0:3]
sample_imgs = sample_imgs1 + sample_imgs2

datamodule = SemanticSegmentationData.from_files(
    predict_files=sample_imgs,
    batch_size=3
)
codes = np.loadtxt(os.path.join(r"/content/data/camvid_tiny","codes.txt"),dtype=str)
class_list = list(codes)
# Define class names
#class_list = ['Animal', 'Archway', 'Bicyclist', 'Bridge', 'Building', 'Car', 'CartLuggagePram', 'Child', 'Column_Pole', 'Fence', 'LaneMkgsDriv', 'LaneMkgsNonDriv', 'Misc_Text', 'MotorcycleScooter', 'OtherMoving', 'ParkingBlock', 'Pedestrian', 'Road', 'RoadShoulder', 'Sidewalk', 'SignSymbol', 'Sky', 'SUVPickupTruck', 'TrafficCone', 'TrafficLight', 'Train', 'Tree', 'Truck_Bus', 'Tunnel', 'VegetationMisc', 'Void', 'Wall']

# Define colors for each class (random example colors)
colors = np.random.randint(0, 255, size=(len(class_list), 3))

def decode_segmap(image, num_classes, colors):
    """Decode the segmentation map back to RGB colors."""
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    for l in range(num_classes):
        idx = image == l
        r[idx] = colors[l, 0]
        g[idx] = colors[l, 1]
        b[idx] = colors[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb

# Create the figure and axes
fig, axarr = plt.subplots(ncols=2, nrows=len(sample_imgs), figsize=(15, 5 * len(sample_imgs)))

# Create a color patch for each class
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=class_name,
                         markerfacecolor=colors[i] / 255, markersize=10) for i, class_name in enumerate(class_list)]

# Add a legend for the classes
fig.legend(handles=legend_elements, loc='center right', bbox_to_anchor=(1.05, 0.5), ncol=1, title="Classes")

running_i = 0
for preds in trainer.predict(model, datamodule=datamodule, output="labels"):
    for pred in preds:
        # Convert pred to a numpy array
        pred_np = np.array(pred)
        decoded_pred = decode_segmap(pred_np, len(class_list), colors)

        img = plt.imread(sample_imgs[running_i])
        axarr[running_i, 0].imshow(img)
        axarr[running_i, 1].imshow(decoded_pred)

        # Hide axes for better visualization
        axarr[running_i, 1].get_xaxis().set_visible(False)
        axarr[running_i, 1].get_yaxis().set_visible(False)
        axarr[running_i, 0].get_xaxis().set_visible(False)
        axarr[running_i, 0].get_yaxis().set_visible(False)

        running_i += 1

# Save the figure with the legend
plt.tight_layout()
plt.savefig("predictions_.png")  # Save all predictions in a single file
plt.show()


Overwriting test.py


In [ ]:
!python test.py

In [ ]:
!pip list > requirements.txt